[View in Colaboratory](https://colab.research.google.com/github/opooladz/ece_210_final_project/blob/master/210_final_project_classify_mod_convnet.ipynb)

In [0]:
# Import Libs 
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math
#from  scipy.ndimage import convolve1d

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# Keras try model 
def try_model(model, data, name, epochs=200, verbose=1):  
  model.compile(optimizer='nadam',
            loss='categorical_crossentropy',
            metrics=[ 'accuracy'])

  model.summary()  
  history = model.fit(data['x_train'], data['y_train'], 
            epochs=epochs, batch_size=64, verbose=verbose, 
            validation_data=(data['x_val'], data['y_val']))  
  score = model.evaluate(data['x_test'], data['y_test'])
  print('Test accuracy: {}'.format(score))
  
#   out_path = 'serialized_models/{}.h5'.format(name)  
#   model.save(out_path)
  
#   with open('serialized_models/{}_history.pickle'.format(name), 'wb') as handle:
#     pickle.dump(history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
  return history

In [0]:
# Comms Stuff Will add more later
def qammod(m,y):
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  # x = np.array([a,1.*b]).T
  return [x , y] 

def qammod2(m,size,trials):
  """
  m - highest integer for the randint
  """
  y = np.random.randint(m,size=(trials,size))
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  #x = a + 1.*1j*b
  # lets hold x in a dp array([real,imag])
  #x = np.array([a,1.*b])
  x = np.dstack((a,b))
  return [x , y]                        

def channel2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  
  [x,y] = qammod2(m,sequenceLen,trials)
  
  h = [1., 0.5, 0.3, -0.13, 0.2, 0, 0.4]
  #h = np.random.randn(1,chanelLen)
  #h = [1]
  #print(x.shape)
#   xh = np.empty((sequenceLen+len(h)-1,2))
#   xh[:,0] = np.convolve(x[:,0],h)   
#   xh[:,1] = np.convolve(x[:,1],h)   
  #xh = convolve1d(x,h,axis = 1)
  xh = np.apply_along_axis(lambda m: np.convolve(m,h[0:chanelLen],mode="full"),axis = 1,arr=x)
  #print(xh.shape)
  SNR = 10**(SNRdB/10)
  sym_noise_pow = np.var(xh)/SNR
  sym_noise_scale_fact = np.sqrt(sym_noise_pow/2)
  if(Noise):
    AWGN = sym_noise_scale_fact*np.random.normal(1, size=xh.shape) 
    xh = xh + AWGN
  #return [xhN.T , np.repeat(y,2) ]
  return [xh , y ]
  
                        
data={}
def generateData(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  [data['x_train'],data['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_val'],data['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data['x_test'],data['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen) 
                        
data2={}
def generateData2(m,sequenceLen,trials,Noise,SNRdB,chanelLen):
  [data2['x_train'],data2['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data2['x_val'],data2['y_val']] = channel2(m=m,sequenceLen=sequenceLen,trials=int(trials*0.5),Noise= Noise,SNRdB=SNRdB,chanelLen=chanelLen)
  [data2['x_test'],data2['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB,chanelLen=chanelLen)


In [0]:
def unison_shuffled_copies(a, b, c):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]

In [11]:
# Visulizing the data

m = 4         # order of modulation
#seqLen = 64    # length of one trial must be square powers of 2 to make sense see below 
trials = 5000   # number of trials
validation_trials = int(trials*0.5)
Noise = 1
SNRdB = 0
chanelLen = 3


x = 4
square_power_of_two = 2**(2*x)
seqLen = square_power_of_two
print(seqLen)

generateData(4,seqLen,trials,Noise,SNRdB,chanelLen)
print(data['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data['y_train'].shape) # labels
data['y_train_QAM'] = np.zeros((trials,1))
data['y_val_QAM'] = np.zeros((validation_trials,1))
data['y_test_QAM'] = np.zeros((trials,1))

generateData2(16,seqLen,trials,Noise,SNRdB,chanelLen)
print(data2['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data2['y_train'].shape) # labels
data2['y_train_QAM'] = np.ones((trials,1))
data2['y_val_QAM'] = np.ones((validation_trials,1))
data2['y_test_QAM'] = np.ones((trials,1))

bigData = {}
bigData['x_train'] = np.reshape(np.stack((data['x_train'],data2['x_train']), axis=0), (trials*2, seqLen+chanelLen-1, 2))
bigData['x_val'] = np.reshape(np.stack((data['x_val'],data2['x_val']), axis=0), (validation_trials * 2, seqLen+chanelLen-1, 2))
bigData['x_test'] = np.reshape(np.stack((data['x_test'],data2['x_test']), axis=0), (trials*2, seqLen+chanelLen-1, 2))

bigData['y_train'] = np.reshape(np.stack((data['y_train'],data2['y_train']), axis=0), (trials*2, seqLen))
bigData['y_val'] = np.reshape(np.stack((data['y_val'],data2['y_val']), axis=0), (validation_trials * 2, seqLen))
bigData['y_test'] = np.reshape(np.stack((data['y_test'],data2['y_test']), axis=0), (trials*2, seqLen))

bigData['y_train_QAM'] = np.reshape(np.stack((data['y_train_QAM'],data2['y_train_QAM']), axis=0), (trials*2,1))
bigData['y_val_QAM'] = np.reshape(np.stack((data['y_val_QAM'],data2['y_val_QAM']), axis=0), (validation_trials*2,1))
bigData['y_test_QAM'] = np.reshape(np.stack((data['y_test_QAM'],data2['y_test_QAM']), axis=0), (trials*2,1))

[bigData['x_train'], bigData['y_train_QAM'], bigData['y_train']] = unison_shuffled_copies(bigData['x_train'], bigData['y_train_QAM'], bigData['y_train'])
[bigData['x_val'], bigData['y_val_QAM'], bigData['y_val']] = unison_shuffled_copies(bigData['x_val'], bigData['y_val_QAM'], bigData['y_val'])
[bigData['x_test'], bigData['y_test_QAM'], bigData['y_test']] = unison_shuffled_copies(bigData['x_test'], bigData['y_test_QAM'], bigData['y_test'])


256
(5000, 258, 2)
(5000, 256)
(5000, 258, 2)
(5000, 256)


In [12]:

# Some keras stuffs
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Reshape, Permute
from keras import regularizers
import keras
ykeys = {'y_train','y_val','y_test'}
xkeys = {'x_train','x_val','x_test'}

  
y_train = keras.utils.to_categorical(bigData['y_train_QAM'] )
y_val = keras.utils.to_categorical(bigData['y_val_QAM'] )
y_test = keras.utils.to_categorical(bigData['y_test_QAM'] )

#y_train = bigData['y_train_QAM']
#y_val = bigData['y_val_QAM']
#y_test = bigData['y_test_QAM']
#print(bigData['x_train'].shape)
#print(bigData['x_val'].shape)
#print(bigData['x_test'].shape)
#print(y_train.shape)
#print(y_val.shape)
#print(y_test.shape)


default_data = {
  'x_train': bigData['x_train'][:,0:bigData['y_train'].shape[1]-chanelLen+1,:],
  'x_val': bigData['x_val'][:,0:bigData['y_val'].shape[1]-chanelLen+1,:],
  'x_test': bigData['x_test'][:,0:bigData['y_test'].shape[1]-chanelLen+1,:],
  'y_train': y_train,
  'y_val': y_val,
  'y_test': y_test,
}
# print(default_data['x_train'].shape)
# print(default_data['x_val'].shape)
# print(default_data['x_test'].shape)
# print(default_data['y_train'].shape)
# print(default_data['y_val'].shape)
# print(default_data['y_test'].shape)


xkeys = {'x_train','x_val','x_test'}
bigDataX = {x:bigData[x].reshape(bigData[x].shape[0],-1,1) for x in xkeys}
default_data.update(bigDataX)
#bigDataY={y:np.expand_dims(default_data[y],axis=2) for y in ykeys}
#default_data.update(bigDataY)
print(default_data['x_train'].shape)
print(default_data['x_test'].shape)
print(default_data['y_train'].shape)
print(default_data['y_test'].shape)
print(default_data['x_val'].shape)
print(default_data['y_val'].shape)

(10000, 516, 1)
(10000, 516, 1)
(10000, 2)
(10000, 2)
(5000, 516, 1)
(5000, 2)


In [13]:
# QAM-4
from keras.layers import Reshape,MaxPooling1D,Dense,Embedding,Conv1D, LSTM,CuDNNLSTM, TimeDistributed, GaussianNoise, Softmax,Flatten
from keras import regularizers
#np.expand_dims()
modCNN = Sequential()
#modCNN.add(Reshape(target_shape = (200,20,1),input_shape=(None,20)))
modCNN.add(Conv1D(40,kernel_size=(10),input_shape=((seqLen+chanelLen-1)*2,1)))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(40,kernel_size=(10)))
modCNN.add(BatchNormalization())
modCNN.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))
modCNN.add(Conv1D(40,kernel_size=(10)))
modCNN.add(BatchNormalization())
modCNN.add(Conv1D(40,kernel_size=(10)))
modCNN.add(BatchNormalization())
modCNN.add(Flatten())
# modCNN.add(Dense(2,bias_regularizer=regularizers.l2(0.005)))
# modCNN.add(Reshape(target_shape = (seqLen,m)))
# modCNN.add(BatchNormalization())
modCNN.add(Dropout(0.7))
#modCNN.add(BatchNormalization())
modCNN.add(Dense(2, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
try_model(modCNN, default_data, 'conv_lstmx', epochs=1)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 507, 40)           440       
_________________________________________________________________
batch_normalization_1 (Batch (None, 507, 40)           160       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 498, 40)           16040     
_________________________________________________________________
batch_normalization_2 (Batch (None, 498, 40)           160       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 249, 40)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 240, 40)           16040     
_________________________________________________________________
batch_normalization_3 (Batch (None, 240, 40)           160       
__________

10000/10000 [==============================] - 2s 232us/step
Test accuracy: [0.022353109589219093, 1.0]
